<a href="https://colab.research.google.com/github/emilio8av/Ejercicios-HTML/blob/main/C%C3%A1lculo_de_%C3%8Dndices_con_herramientas_generativas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imágen NDVI con herramientas generativas**

In [ ]:
!pip install rasterio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 57.2 MB/s eta 0:00:00


In [ ]:
# Función principal para analizar imágenes Sentinel-2
def procesar_ndvi(ruta_b4=None, ruta_b8=None, esquema='basico', mostrar_estadisticas=True):
    """
    Procesa imágenes Sentinel-2 para calcular y clasificar NDVI.

    Args:
        ruta_b4: Ruta al archivo de banda roja (B4)
        ruta_b8: Ruta al archivo de banda infrarroja (B8)
        esquema: 'basico' (4 clases) o 'detallado' (6 clases)
        mostrar_estadisticas: Mostrar gráfico de estadísticas junto al mapa

    Returns:
        Diccionario con resultados del procesamiento
    """
    try:
        # Verificar archivos
        if not ruta_b4 or not ruta_b8:
            print("Error: Debe proporcionar rutas a los archivos de banda roja (B4) e infrarroja (B8).")
            return None

        ruta_b4 = Path(ruta_b4)
        ruta_b8 = Path(ruta_b8)

        if not ruta_b4.exists() or not ruta_b8.exists():
            print(f"Error: Uno o ambos archivos no existen.")
            return None

        print(f"Procesando:")
        print(f"- Banda 4 (roja): {ruta_b4}")
        print(f"- Banda 8 (NIR): {ruta_b8}")

        # Cargar bandas
        print("Cargando bandas Sentinel-2...")
        rojo, nir, perfil = cargar_bandas(ruta_b4, ruta_b8)

        # Calcular NDVI
        print("Calculando índice NDVI...")
        ndvi = calcular_ndvi(rojo, nir)

        # Obtener esquema de clasificación
        esquema_detallado = (esquema == 'detallado')
        clases_ndvi = crear_esquema_clasificacion_ndvi(personalizado=esquema_detallado)

        # Clasificar NDVI
        print(f"Clasificando cobertura según valores NDVI (esquema {esquema})...")
        ndvi_clasificado, clases_usadas, estadisticas = clasificar_ndvi_mejorado(ndvi, clases_ndvi)

        # Extraer el nombre base de los archivos de entrada para usarlo en los archivos de salida
        nombre_base = Path(ruta_b4).stem
        if '_B04' in nombre_base or '_B4' in nombre_base:
            nombre_base = nombre_base.replace('_B04', '').replace('_B4', '')

        # Definir rutas de salida en el directorio actual para fácil descarga
        ruta_reporte = Path(f'./{nombre_base}_ndvi_reporte.txt')
        ruta_salida_png = Path(f'./{nombre_base}_ndvi_clasificacion.png')
        ruta_tif = Path(f'./{nombre_base}_ndvi_clasificacion.tif')

        # Generar reporte
        generar_reporte_estadisticas(estadisticas, clases_usadas, ruta_reporte)

        # Generar gráfica
        print("Generando gráfica para publicación...")
        fig = generar_grafica_publicacion(ndvi_clasificado, clases_usadas, estadisticas,
                                         ruta_salida_png, mostrar_estadisticas=mostrar_estadisticas)

        # Guardar raster clasificado
        with rasterio.open(str(ruta_tif), 'w',
                          driver='GTiff',
                          height=ndvi_clasificado.shape[0],
                          width=ndvi_clasificado.shape[1],
                          count=1,
                          dtype=np.uint8,
                          crs=perfil['crs'],
                          transform=perfil['transform']) as dst:
            dst.write(ndvi_clasificado.astype(np.uint8), 1)
        print(f"Raster clasificado guardado en: {ruta_tif}")

        print("\nPara descargar los archivos generados:")
        print("from google.colab import files")
        print(f"files.download('{nombre_base}_ndvi_clasificacion.png')")
        print(f"files.download('{nombre_base}_ndvi_reporte.txt')")
        print(f"files.download('{nombre_base}_ndvi_clasificacion.tif')")

        # Devolver resultados para seguir trabajando
        return {
            'ndvi': ndvi,
            'ndvi_clasificado': ndvi_clasificado,
            'estadisticas': estadisticas,
            'clases': clases_usadas,
            'figura': fig
        }

    except Exception as e:
        print(f"Error durante el procesamiento: {e}")
        import traceback
        traceback.print_exc()
        return None

# PARTE 1: Carga de archivos
# ==========================
# Cargar archivos directamente a Colab
print("Cargue el archivo de banda roja (B4) de Sentinel-2:")
uploaded_b4 = files.upload()  # Esto abre un diálogo para subir archivos

if uploaded_b4:
    ruta_b4 = next(iter(uploaded_b4.keys()))
    print(f"Archivo cargado: {ruta_b4}")

    print("\nAhora cargue el archivo de banda infrarroja (B8) de Sentinel-2:")
    uploaded_b8 = files.upload()

    if uploaded_b8:
        ruta_b8 = next(iter(uploaded_b8.keys()))
        print(f"Archivo cargado: {ruta_b8}")
    else:
        print("No se cargó el archivo de banda infrarroja.")
else:
    print("No se cargó el archivo de banda roja.")

# PARTE 2: Procesamiento
# ======================
# Verificar si tenemos los archivos necesarios
if 'ruta_b4' in locals() and 'ruta_b8' in locals():
    print("\nSeleccione el esquema de clasificación:")
    print("1. Básico (4 clases)")
    print("2. Detallado (6 clases)")
    opcion_esquema = input()
    esquema = 'detallado' if opcion_esquema == '2' else 'basico'

    print("\nProcesando imágenes Sentinel-2...")
    resultados = procesar_ndvi(ruta_b4=ruta_b4, ruta_b8=ruta_b8, esquema=esquema)

    if resultados:
        print("\nProcesamiento completado con éxito.")

        # Análisis adicional (opcional)
        print("\n¿Desea realizar un análisis adicional del NDVI? (s/n)")
        analisis_adicional = input()

        if analisis_adicional.lower() == 's':
            # Mostrar histograma del NDVI
            plt.figure(figsize=(10, 6))
            plt.hist(resultados['ndvi'].flatten(), bins=50, range=(-1, 1), alpha=0.7)
            plt.title('Histograma de valores NDVI', fontsize=14)
            plt.xlabel('Valor NDVI', fontsize=12)
            plt.ylabel('Frecuencia', fontsize=12)
            plt.grid(alpha=0.3)
            plt.show()

            # Mostrar estadísticas básicas
            ndvi_valid = resultados['ndvi'][~np.isnan(resultados['ndvi'])]
            print("\nEstadísticas básicas del NDVI:")
            print(f"Media: {np.mean(ndvi_valid):.4f}")
            print(f"Mediana: {np.median(ndvi_valid):.4f}")
            print(f"Desviación estándar: {np.std(ndvi_valid):.4f}")
            print(f"Mínimo: {np.min(ndvi_valid):.4f}")
            print(f"Máximo: {np.max(ndvi_valid):.4f}")

            # Extraer el nombre base para las instrucciones de descarga
            nombre_base = Path(ruta_b4).stem
            if '_B04' in nombre_base or '_B4' in nombre_base:
                nombre_base = nombre_base.replace('_B04', '').replace('_B4', '')

            # Descargar resultados
            print("\nPara descargar los resultados, ejecute:")
            print("from google.colab import files")
            print(f"files.download('{nombre_base}_ndvi_clasificacion.png')")
            print(f"files.download('{nombre_base}_ndvi_reporte.txt')")
            print(f"files.download('{nombre_base}_ndvi_clasificacion.tif')")
else:
    print("\nNo se han especificado los archivos necesarios para el procesamiento.")
    print("Por favor, ejecute nuevamente este notebook y cargue los archivos requeridos.")

# Nota: Este notebook es un ejemplo interactivo para procesar imágenes Sentinel-2.rámetros según sus necesidades.# Clasificación NDVI con Google Colab
# ===========================
# Este notebook muestra cómo procesar imágenes Sentinel-2 para calcular y clasificar el índice NDVI.

# Instalar las bibliotecas necesarias si no están instaladas
!pip install rasterio matplotlib numpy

# Importar las bibliotecas
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib import rcParams
from pathlib import Path
import os
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
from datetime import datetime
from google.colab import files, drive

# Configurar parámetros de matplotlib para publicación académica
rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 10
rcParams['axes.linewidth'] = 1.0
rcParams['axes.labelsize'] = 11
rcParams['xtick.labelsize'] = 10
rcParams['ytick.labelsize'] = 10
rcParams['legend.fontsize'] = 10
rcParams['figure.dpi'] = 300

# Montar Google Drive (opcional, si tus archivos están allí)
# Descomenta las siguientes líneas si quieres usar archivos desde tu Google Drive
# drive.mount('/content/drive')
# print("Google Drive montado en /content/drive")

# Definición de la clase NDVI y funciones auxiliares
@dataclass
class ClaseNDVI:
    """Clase para definir una categoría de clasificación de NDVI."""
    id: int
    nombre: str
    valor_min: float
    valor_max: float
    color: str
    descripcion: Optional[str] = None

def cargar_bandas(ruta_banda_roja, ruta_banda_nir):
    """
    Carga las bandas roja e infrarroja cercana.

    Args:
        ruta_banda_roja: Ruta al archivo de la banda roja (B4)
        ruta_banda_nir: Ruta al archivo de la banda infrarroja cercana (B8)

    Returns:
        Tuple con arrays de banda roja, banda infrarroja y perfil del raster
    """
    # Convertir a Path si se proporciona como string
    ruta_banda_roja = Path(ruta_banda_roja)
    ruta_banda_nir = Path(ruta_banda_nir)

    # Verificar existencia de archivos
    if not ruta_banda_roja.exists():
        raise FileNotFoundError(f"Archivo de banda roja no encontrado: {ruta_banda_roja}")
    if not ruta_banda_nir.exists():
        raise FileNotFoundError(f"Archivo de banda infrarroja no encontrado: {ruta_banda_nir}")

    # Abrir y leer archivos
    with rasterio.open(str(ruta_banda_roja)) as red_src:
        red = red_src.read(1).astype('float32')
        profile = red_src.profile.copy()

    with rasterio.open(str(ruta_banda_nir)) as nir_src:
        nir = nir_src.read(1).astype('float32')

        # Verificar compatibilidad de dimensiones
        if nir.shape != red.shape:
            raise ValueError(f"Las dimensiones de las bandas no coinciden: Roja {red.shape}, NIR {nir.shape}")

    return red, nir, profile

def calcular_ndvi(banda_roja, banda_nir):
    """
    Calcula el índice de vegetación NDVI.

    Args:
        banda_roja: Array numpy de la banda roja
        banda_nir: Array numpy de la banda infrarroja cercana

    Returns:
        Array numpy con los valores NDVI
    """
    # Verificar datos válidos
    if np.all(banda_roja <= 0) or np.all(banda_nir <= 0):
        print("Advertencia: Las bandas parecen contener valores muy bajos o negativos.")

    # Crear una máscara para proteger contra la división por cero
    denominador = banda_nir + banda_roja
    mascara = denominador > 0  # Evitar división por cero

    # Inicializar array NDVI con -1 (valor para agua/sombras)
    ndvi = np.full_like(banda_roja, -1, dtype=np.float32)

    # Calcular NDVI solo donde el denominador no es cero
    ndvi[mascara] = (banda_nir[mascara] - banda_roja[mascara]) / denominador[mascara]

    # Asegurar que los valores estén en el rango [-1, 1]
    ndvi = np.clip(ndvi, -1.0, 1.0)

    return ndvi

def crear_esquema_clasificacion_ndvi(personalizado=False):
    """
    Crea un esquema de clasificación para NDVI.

    Args:
        personalizado: Si es True, devuelve un esquema personalizado más detallado.
                      Si es False, usa el esquema básico de 4 clases del código original.

    Returns:
        Lista de objetos ClaseNDVI con la definición del esquema de clasificación
    """
    if not personalizado:
        # Esquema básico de 4 clases (como en el código original)
        return [
            ClaseNDVI(id=1, nombre="Agua/Sombras", valor_min=-1.0, valor_max=0.0,
                     color='#2c7bb6', descripcion="Cuerpos de agua, sombras y nubes"),
            ClaseNDVI(id=2, nombre="Suelo desnudo", valor_min=0.0, valor_max=0.2,
                     color='#ffda66', descripcion="Áreas urbanas, suelo expuesto, rocas"),
            ClaseNDVI(id=3, nombre="Vegetación herbácea", valor_min=0.2, valor_max=0.4,
                     color='#90cc6f', descripcion="Vegetación de baja densidad, pastizales"),
            ClaseNDVI(id=4, nombre="Vegetación densa", valor_min=0.4, valor_max=1.0,
                     color='#00441b', descripcion="Bosques, cultivos densos")
        ]
    else:
        # Esquema más detallado (6 clases) para análisis más fino
        return [
            ClaseNDVI(id=1, nombre="Agua profunda", valor_min=-1.0, valor_max=-0.3,
                     color='#0f2080', descripcion="Agua profunda, sombras densas"),
            ClaseNDVI(id=2, nombre="Agua superficial", valor_min=-0.3, valor_max=0.0,
                     color='#4dac26', descripcion="Agua superficial, zonas húmedas"),
            ClaseNDVI(id=3, nombre="Suelo desnudo", valor_min=0.0, valor_max=0.1,
                     color='#b35806', descripcion="Suelo expuesto, áreas urbanas densas"),
            ClaseNDVI(id=4, nombre="Área urbana/rocas", valor_min=0.1, valor_max=0.2,
                     color='#f1a340', descripcion="Baja reflectancia, zonas urbanas, rocas"),
            ClaseNDVI(id=5, nombre="Vegetación dispersa", valor_min=0.2, valor_max=0.4,
                     color='#92c37d', descripcion="Vegetación de baja densidad, matorrales"),
            ClaseNDVI(id=6, nombre="Vegetación densa", valor_min=0.4, valor_max=1.0,
                     color='#1b7837', descripcion="Bosques densos, cultivos saludables")
        ]

def clasificar_ndvi_mejorado(ndvi_array, clases_ndvi=None):
    """
    Versión mejorada de la clasificación de NDVI, con mayor flexibilidad
    y mejor manejo de casos límite.

    Args:
        ndvi_array: Array numpy con valores NDVI
        clases_ndvi: Lista de objetos ClaseNDVI (si es None, se usa el esquema básico)

    Returns:
        Tuple con (array clasificado, lista de clases usadas, estadísticas)
    """
    # Si no se proporcionan clases, usar el esquema predeterminado
    if clases_ndvi is None:
        clases_ndvi = crear_esquema_clasificacion_ndvi()

    # Crear una máscara para valores no válidos fuera del rango [-1, 1]
    mascara_valida = (ndvi_array >= -1.0) & (ndvi_array <= 1.0) & (~np.isnan(ndvi_array))

    # Inicializar array con valor no válido (0)
    ndvi_clasificado = np.zeros_like(ndvi_array, dtype=np.uint8)

    # Aplicar clasificación con bucle explícito para mejor claridad
    for clase in clases_ndvi:
        # Para la última clase, incluimos el valor máximo
        if clase.id == len(clases_ndvi):
            mascara = (ndvi_array >= clase.valor_min) & (ndvi_array <= clase.valor_max) & mascara_valida
        else:
            mascara = (ndvi_array >= clase.valor_min) & (ndvi_array < clase.valor_max) & mascara_valida

        ndvi_clasificado[mascara] = clase.id

    # Calcular estadísticas de la clasificación (para fines informativos)
    estadisticas = {}
    total_pixeles = np.sum(mascara_valida)

    for clase in clases_ndvi:
        pixeles_clase = np.sum(ndvi_clasificado == clase.id)
        porcentaje = (pixeles_clase / total_pixeles) * 100 if total_pixeles > 0 else 0
        estadisticas[clase.nombre] = {
            'pixeles': int(pixeles_clase),
            'porcentaje': float(porcentaje)
        }

    return ndvi_clasificado, clases_ndvi, estadisticas

def generar_grafica_publicacion(ndvi_clasificado, clases_ndvi, estadisticas=None,
                               ruta_salida=None, dpi=600, mostrar_estadisticas=True):
    """
    Genera una gráfica de alta calidad para publicación académica.

    Args:
        ndvi_clasificado: Array numpy con valores NDVI clasificados
        clases_ndvi: Lista de objetos ClaseNDVI usados en la clasificación
        estadisticas: Diccionario con estadísticas de la clasificación
        ruta_salida: Ruta para guardar la imagen (si es None, solo muestra)
        dpi: Resolución de la imagen para guardar
        mostrar_estadisticas: Si es True, incluye un gráfico de barras con estadísticas
    """
    # Extraer colores, etiquetas y IDs de las clases
    colores = [clase.color for clase in clases_ndvi]
    etiquetas = [clase.nombre for clase in clases_ndvi]
    ids = [clase.id for clase in clases_ndvi]

    # Crear mapa de colores y normalización
    cmap = mcolors.ListedColormap(colores)
    # Creamos límites que son 0.5 unidades por debajo y por encima del rango de IDs
    bounds = [id-0.5 for id in ids] + [ids[-1]+0.5]
    norm = mcolors.BoundaryNorm(bounds, cmap.N)

    # Decidir si crear una figura con o sin gráfico de estadísticas
    if mostrar_estadisticas and estadisticas:
        # Crear figura con dos subplots: mapa y estadísticas
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5.5),
                                      gridspec_kw={'width_ratios': [3, 1]})

        # Subplot 1: Mapa de clasificación
        img = ax1.imshow(ndvi_clasificado, cmap=cmap, norm=norm, interpolation='none')

        # Eliminar ejes y labels del mapa
        ax1.set_xticks([])
        ax1.set_yticks([])
        ax1.spines['top'].set_visible(False)
        ax1.spines['right'].set_visible(False)
        ax1.spines['bottom'].set_visible(False)
        ax1.spines['left'].set_visible(False)

        # Añadir título al mapa
        ax1.set_title('Clasificación de cobertura por NDVI', fontsize=12, fontweight='bold')

        # Añadir barra de escala (simulada) - ajustar según dimensiones de la imagen
        scale_bar_length = min(ndvi_clasificado.shape) // 10  # Longitud proporcional al tamaño de la imagen
        scale_position_x = ndvi_clasificado.shape[1] // 20
        scale_position_y = ndvi_clasificado.shape[0] - ndvi_clasificado.shape[0] // 10

        ax1.plot([scale_position_x, scale_position_x + scale_bar_length],
                [scale_position_y, scale_position_y], 'k-', lw=2)
        ax1.text(scale_position_x + scale_bar_length // 2, scale_position_y + 5,
                '1 km', ha='center', va='top', fontsize=9)

        # Añadir indicador de norte
        north_position_x = scale_position_x
        north_position_y = scale_position_y - 30
        ax1.text(north_position_x, north_position_y, '↑N', ha='center',
                va='center', fontsize=12, fontweight='bold')

        # Subplot 2: Gráfico de barras con estadísticas
        porcentajes = [estadisticas[clase.nombre]['porcentaje'] for clase in clases_ndvi]
        y_pos = range(len(etiquetas))

        # Crear barras horizontales con los colores de las clases
        bars = ax2.barh(y_pos, porcentajes, color=colores)

        # Añadir porcentajes como etiquetas
        for i, bar in enumerate(bars):
            width = bar.get_width()
            label_x_pos = width + 1 if width > 10 else width + 3
            ax2.text(label_x_pos, bar.get_y() + bar.get_height()/2,
                   f'{porcentajes[i]:.1f}%',
                   va='center', fontsize=9)

        # Configurar ejes y etiquetas
        ax2.set_yticks(y_pos)
        ax2.set_yticklabels(etiquetas)
        ax2.set_xlabel('Porcentaje del área (%)')
        ax2.set_title('Distribución de clases', fontsize=11)
        ax2.set_xlim(0, max(porcentajes) * 1.2)  # Dar espacio para las etiquetas

        # Añadir cuadrícula al gráfico de barras para facilitar la lectura
        ax2.grid(axis='x', linestyle='--', alpha=0.7)

        # Alinear y ajustar
        plt.tight_layout()

    else:
        # Crear figura solo con el mapa
        fig, ax = plt.subplots(figsize=(7.5, 5.5))
        img = ax.imshow(ndvi_clasificado, cmap=cmap, norm=norm, interpolation='none')

        # Eliminar ejes y labels
        ax.set_xticks([])
        ax.set_yticks([])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)

        # Añadir barra de escala y norte - ajustados dinámicamente
        scale_bar_length = min(ndvi_clasificado.shape) // 10
        scale_position_x = ndvi_clasificado.shape[1] // 20
        scale_position_y = ndvi_clasificado.shape[0] - ndvi_clasificado.shape[0] // 10

        ax.plot([scale_position_x, scale_position_x + scale_bar_length],
               [scale_position_y, scale_position_y], 'k-', lw=2)
        ax.text(scale_position_x + scale_bar_length // 2, scale_position_y + 5,
               '1 km', ha='center', va='top', fontsize=9)

        north_position_x = scale_position_x
        north_position_y = scale_position_y - 30
        ax.text(north_position_x, north_position_y, '↑N', ha='center',
               va='center', fontsize=12, fontweight='bold')

        # Crear leyenda personalizada con parches de color
        patches = [mpatches.Patch(color=clase.color, label=clase.nombre)
                  for clase in clases_ndvi]
        ax.legend(handles=patches, loc='lower right', frameon=True,
                 framealpha=0.9, edgecolor='lightgray', fontsize=9)

        # Añadir título y subtítulo
        ax.set_title('Clasificación de cobertura del suelo basada en NDVI',
                    fontsize=12, fontweight='bold', pad=10)
        ax.text(ndvi_clasificado.shape[1]/2, -20,
               'Sentinel-2, Catamayo (Ecuador), Noviembre 2017',
               ha='center', va='top', fontsize=9, style='italic')

    # Añadir información de la fuente en la esquina inferior izquierda
    plt.annotate('Fuente: Elaboración propia con datos Sentinel-2 MSI',
                xy=(0.01, 0.01), xycoords='figure fraction',
                fontsize=7, color='dimgray')

    # Guardar si se especifica ruta
    if ruta_salida:
        # Convertir a Path si es string
        ruta_salida = Path(ruta_salida)

        # Asegurar que el directorio exista
        if not ruta_salida.parent.exists():
            ruta_salida.parent.mkdir(parents=True, exist_ok=True)

        # Guardar con alta resolución
        plt.savefig(str(ruta_salida), dpi=dpi, bbox_inches='tight')
        print(f"Figura guardada en: {ruta_salida}")

        # También guardar en formato vectorial para la publicación
        if ruta_salida.suffix == '.png':
            ruta_vectorial = ruta_salida.with_suffix('.pdf')
            plt.savefig(str(ruta_vectorial), format='pdf', bbox_inches='tight')
            print(f"Versión vectorial guardada en: {ruta_vectorial}")

    # Mostrar la figura
    plt.show()

    return fig

def generar_reporte_estadisticas(estadisticas, clases_ndvi, ruta_salida=None):
    """
    Genera un reporte de texto con las estadísticas de la clasificación.

    Args:
        estadisticas: Diccionario con estadísticas de la clasificación
        clases_ndvi: Lista de objetos ClaseNDVI usados en la clasificación
        ruta_salida: Ruta para guardar el reporte (si es None, solo imprime)
    """
    # Crear el reporte
    reporte = ["REPORTE DE CLASIFICACIÓN NDVI", "=" * 30, ""]
    reporte.append(f"Fecha de procesamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    reporte.append("Clases utilizadas:")

    # Tabla de clases
    reporte.append("\nID | Nombre              | Rango NDVI      | Descripción")
    reporte.append("-" * 70)

    for clase in clases_ndvi:
        id_str = str(clase.id).ljust(3)
        nombre_str = clase.nombre.ljust(20)
        rango_str = f"[{clase.valor_min:.1f}, {clase.valor_max:.1f}]".ljust(15)
        desc_str = clase.descripcion if clase.descripcion else ""
        reporte.append(f"{id_str}| {nombre_str}| {rango_str}| {desc_str}")

    # Estadísticas
    reporte.append("\nEstadísticas de la clasificación:")
    reporte.append("-" * 50)
    reporte.append("Clase                | Píxeles     | Porcentaje")
    reporte.append("-" * 50)

    total_pixeles = sum(estadisticas[clase.nombre]['pixeles'] for clase in clases_ndvi)

    for clase in clases_ndvi:
        stats = estadisticas[clase.nombre]
        nombre_str = clase.nombre.ljust(20)
        pixeles_str = str(stats['pixeles']).rjust(12)
        pct_str = f"{stats['porcentaje']:.2f}%".rjust(10)
        reporte.append(f"{nombre_str}| {pixeles_str} | {pct_str}")

    reporte.append("-" * 50)
    reporte.append(f"Total               | {total_pixeles:12d} | 100.00%")

    # Unir todas las líneas
    reporte_texto = "\n".join(reporte)

    # Imprimir o guardar
    print(reporte_texto)

    if ruta_salida:
        # Convertir a Path si es string
        ruta_salida = Path(ruta_salida)

        # Asegurar que el directorio exista
        if not ruta_salida.parent.exists():
            ruta_salida.parent.mkdir(parents=True, exist_ok=True)

        with open(str(ruta_salida), 'w', encoding='utf-8') as f:
            f.write(reporte_texto)
        print(f"Reporte guardado en: {ruta_salida}")

    return reporte_texto